# 1. Extract "Land price per floor area" from html file

In [1]:
import pandas as pd
from datetime import datetime

def grab_highest_bid(line):
    index_start = line.find(">") + len(">")
    index_end = line.find("</td>")
    info = line[index_start: index_end]
    info = info.replace(",", "")
    info = round(float(info))
    return info

def grab_date_info(line):
    index_start = line.find("Sold on") + len("Sold on") + 1
    index_end = line.find("</h3>")
    info = line[index_start: index_end]    
    month_dict = {"Jan":1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

    index_start = info.find(" ") + 1
    index_end = info.find(" ", index_start + 2)
    month = info[index_start: index_end]
    year = info[index_end + 1: ]
    
    info = datetime(int(year), month_dict[month], 1)
    return info

def grab_proj_no_nick(line):
    index_temp = line.find("""<a href="https://www.squarefoot.com.sg/trends-and-analysis/""") + 1
    index_start = line.find(">", index_temp) + len(">")
    index_end = line.find("</a>")
    info = line[index_start: index_end]
    
    if "(" in info:
        index_end = info.find(" (")
        info = info[0: index_end]
    return info

def grab_proj_has_nick(line):
    index_temp = line.find("""Now known as <a href="https://www.squarefoot.com.sg/trends-and-analysis/""")
    
    if (index_temp != -1):
        index_start = line.find(">", index_temp) + len(">")
        index_end = line.find("</a>", index_temp)
        info = line[index_start: index_end]
    else:
        index_start = line.find("Now known as") + len("Now known as ")
        index_end = line.find(")")
        info = line[index_start: index_end]
    
    return info

# retrieve land transaction data from html file, include [project name], [land txn date], [land highest bid]
def retrieve_from_html():
    file = open("Land Sales - Square Foot Research Singapore.html")
    trans_date = []
    trans_proj = []
    trans_bid = []
    proj_dict = []
    all_line_count = 0
    useful_name_count = 0
    useful_bid_count = 0
    
    line = file.readline()
    while line:
        line = file.readline()
        all_line_count += 1
        
        # grab transaction date
        if ("""<td><h3 class="report-title" style="color:#cc3333">""" in line):
            trans_date.append(grab_date_info(line))
            useful_name_count = all_line_count  # make sure a date <=> a transaction
        
        #grab project name
        if (all_line_count - useful_name_count == 1) & ("""<td><h3 class="report-title"><a href="https://www.squarefoot.com.sg/trends-and-analysis/land-sales?""" in line):
            if ("""Now known as""" not in line):
                trans_proj.append(grab_proj_no_nick(line))
            else:
                trans_proj.append(grab_proj_has_nick(line))
                
        #grab highest bid
        if "Highest bid ($psf)" in line:
            useful_bid_count = all_line_count
        if (all_line_count - useful_bid_count == 1) & (useful_bid_count - useful_name_count == 27) :
            trans_bid.append(grab_highest_bid(line))

    file.close()
    
    counter = 0
    for i in range(len(trans_proj)):
        if (trans_proj[i] != "") & (trans_bid[i] != '0.0'):
            counter += 1
            proj_dict.append([trans_date[i], trans_proj[i], trans_bid[i]])

    return proj_dict 

retrieve_from_html()

[[datetime.datetime(2022, 1, 1, 0, 0), 'Gloria Mansions', 1098],
 [datetime.datetime(2021, 12, 1, 0, 0), 'Former High Point', 2626],
 [datetime.datetime(2021, 12, 1, 0, 0), 'Former La Ville', 1540],
 [datetime.datetime(2021, 10, 1, 0, 0), 'Watten Estate Condominium', 1723],
 [datetime.datetime(2021, 10, 1, 0, 0), 'Slim Barracks Rise', 1210],
 [datetime.datetime(2021, 10, 1, 0, 0), 'Slim Barracks Rise', 1246],
 [datetime.datetime(2021, 9, 1, 0, 0), 'Flynn Park', 1318],
 [datetime.datetime(2021, 8, 1, 0, 0), 'Tampines Street 62', 659],
 [datetime.datetime(2021, 6, 1, 0, 0), 'Tengah Garden Walk', 603],
 [datetime.datetime(2021, 6, 1, 0, 0), 'Ang Mo Kio Avenue 1', 1118],
 [datetime.datetime(2021, 6, 1, 0, 0), 'Ji Liang Gardens', 0],
 [datetime.datetime(2021, 5, 1, 0, 0), 'Northumberland Road Condo', 1129],
 [datetime.datetime(2021, 7, 1, 0, 0), '6A/B/C Robin Drive', 1454],
 [datetime.datetime(2021, 3, 1, 0, 0), '2, 4, 6 Mount Emily Road', 1115],
 [datetime.datetime(2021, 3, 1, 0, 0), 'Surr

# 2. match two data resources' project name

In [3]:
def open_file():
    global df_trans, df_proj, df_proj_names, df_proj_ids, df_trans_proj_ids, df_trans_saletypes, df_trans_prices, df_trans_areas,df_trans_dates
    
    df_trans = pd.read_excel(r"transaction.xlsx")
    df_proj = pd.read_excel(r"project.xlsx")
    
    df_proj_names = df_proj["name"]
    df_proj_ids = df_proj["project_id"]

    df_trans_proj_ids = df_trans["project_project_id"]
    df_trans_saletypes = df_trans["sale_type"]
    df_trans_prices = df_trans["price"]
    df_trans_areas = df_trans["floor_area"]
    df_trans_dates = df_trans["contract_date"]

open_file()

In [8]:
def match_proj_name(proj_name_temp):
    proj_name = []
    proj_name_no = []
    df = pd.read_excel(r"project.xlsx")
    proj_name_ori = df["name"].tolist()
    
    for i in proj_name_temp:
        match = False
        for j in proj_name_ori:
            if (i[1].lower()== str(j).lower()):
                proj_name.append(i)
                match = True
                break
                
    return proj_name

# 3. Extract "Property opening price per floor area" from csv file

In [9]:
def get_ave_newsale_price_n_date(proj_name):

    price_sum = 0
    area_sum = 0
    proj_id = -1
    price_n_date = []
    all_dates = []
    
    global invalid_data_index
    
    for i in range(len(df_proj_names)):
        if (proj_name.lower() == str(df_proj_names[i]).lower()):
            proj_id = df_proj_ids[i]

    for j in range(len(df_trans_proj_ids)):
        if (df_trans_proj_ids[j] == proj_id) & (df_trans_saletypes[j] == 1):
            price_sum += df_trans_prices[j]
            area_sum += df_trans_areas[j]
            all_dates.append(df_trans_dates[j])
    
    if (area_sum != 0):
        price_n_date.append(price_sum/ area_sum * 0.093)
    else:
        price_n_date.append(-1)
    
    if (all_dates != []):
        earliest_date = find_earilest_date(all_dates)
        price_n_date.append(earliest_date)
    else:
        price_n_date.append(None)
        
    return price_n_date

In [10]:
def find_earilest_date(all_dates):
    year = -1
    month = -1
    all_year = []
    all_month = []
    new_sale_date = []
        
    for i in all_dates:
        year = i % 100
        month = i // 100
        all_year.append(year)
        all_month.append(month)
    
    all_min_months = []
    for j in range(len(all_year)):
        if all_year[j] == min(all_year):
            all_min_months.append(all_month[j])
            
    date = datetime(min(all_year) + 2000, min(all_min_months), 1)

    return date

# 4. Collate final model trainging dataset and save as csv file

In [11]:
def get_final_data():
    
    all_projs = retrieve_from_html()
    new_projs = all_projs[:16]
    old_projs = all_projs[16:]
    old_projs = match_proj_name(old_projs)
    new_sale_price = []
    new_sale_date = []
    for p in old_projs:
        new_sale_price.append(get_ave_newsale_price_n_date(p[1])[0])
        new_sale_date.append(get_ave_newsale_price_n_date(p[1])[1])
        
    for i in range(len(old_projs)):
        old_projs[i].append(new_sale_date[i])
        old_projs[i].append(round(new_sale_price[i]))
    
    for j in old_projs:
        if (j[3] == -1):
            old_projs.remove(j)

    print(old_projs)

In [14]:
def save_file():
    df_final_data = pd.DataFrame(get_final_data())
    print(df_final_data)

save_file()

[[datetime.datetime(2020, 3, 1, 0, 0), 'Parc Greenwich', 555, datetime.datetime(2021, 9, 1, 0, 0), 1227], [datetime.datetime(2020, 1, 1, 0, 0), 'Irwell Hill Residences', 1515, datetime.datetime(2021, 4, 1, 0, 0), 2672], [datetime.datetime(2020, 1, 1, 0, 0), 'Bartley Vue', 885, datetime.datetime(2021, 9, 1, 0, 0), 1902], [datetime.datetime(2019, 9, 1, 0, 0), 'One-North Eden', 1002, datetime.datetime(2021, 4, 1, 0, 0), 1966], [datetime.datetime(2019, 9, 1, 0, 0), 'Midtown Modern', 1535, datetime.datetime(2021, 3, 1, 0, 0), 2726], [datetime.datetime(2019, 10, 1, 0, 0), 'Provence Residence', 566, datetime.datetime(2021, 5, 1, 0, 0), 1169], [datetime.datetime(2019, 7, 1, 0, 0), 'Clavon', 788, datetime.datetime(2020, 12, 1, 0, 0), 1637], [datetime.datetime(2019, 4, 1, 0, 0), 'The M', 1458, datetime.datetime(2020, 2, 1, 0, 0), 2484], [datetime.datetime(2019, 4, 1, 0, 0), 'Penrose', 732, datetime.datetime(2020, 9, 1, 0, 0), 1604], [datetime.datetime(2019, 12, 1, 0, 0), 'Casa Sophia', 1200, Non

In [20]:
import datetime
all =[[datetime.datetime(2020, 3, 1, 0, 0), 'Parc Greenwich', 555, datetime.datetime(2021, 9, 1, 0, 0), 1227], [datetime.datetime(2020, 1, 1, 0, 0), 'Irwell Hill Residences', 1515, datetime.datetime(2021, 4, 1, 0, 0), 2672], [datetime.datetime(2020, 1, 1, 0, 0), 'Bartley Vue', 885, datetime.datetime(2021, 9, 1, 0, 0), 1902], [datetime.datetime(2019, 9, 1, 0, 0), 'One-North Eden', 1002, datetime.datetime(2021, 4, 1, 0, 0), 1966], [datetime.datetime(2019, 9, 1, 0, 0), 'Midtown Modern', 1535, datetime.datetime(2021, 3, 1, 0, 0), 2726], [datetime.datetime(2019, 10, 1, 0, 0), 'Provence Residence', 566, datetime.datetime(2021, 5, 1, 0, 0), 1169], [datetime.datetime(2019, 7, 1, 0, 0), 'Clavon', 788, datetime.datetime(2020, 12, 1, 0, 0), 1637], [datetime.datetime(2019, 4, 1, 0, 0), 'The M', 1458, datetime.datetime(2020, 2, 1, 0, 0), 2484], [datetime.datetime(2019, 4, 1, 0, 0), 'Penrose', 732, datetime.datetime(2020, 9, 1, 0, 0), 1604], [datetime.datetime(2019, 12, 1, 0, 0), 'Casa Sophia', 1200, None, -1], [datetime.datetime(2018, 2, 1, 0, 0), 'Cairnhill Mansions', 2311, None, -1], [datetime.datetime(2018, 9, 1, 0, 0), 'Ola', 576, datetime.datetime(2020, 3, 1, 0, 0), 1152], [datetime.datetime(2018, 9, 1, 0, 0), 'Verdale', 1002, datetime.datetime(2020, 9, 1, 0, 0), 1765], [datetime.datetime(2018, 9, 1, 0, 0), 'Dairy Farm Residences', 830, datetime.datetime(2019, 11, 1, 0, 0), 1606], [datetime.datetime(2018, 9, 1, 0, 0), 'Parc Canberra', 558, datetime.datetime(2020, 2, 1, 0, 0), 1100], [datetime.datetime(2018, 6, 1, 0, 0), 'Verticus', 1111, datetime.datetime(2020, 2, 1, 0, 0), 2022], [datetime.datetime(2018, 6, 1, 0, 0), 'Teck Guan Ville', 1300, None, -1], [datetime.datetime(2018, 6, 1, 0, 0), 'Park House', 2910, None, -1], [datetime.datetime(2018, 5, 1, 0, 0), 'Neu at Novena', 1472, datetime.datetime(2019, 10, 1, 0, 0), 2607], [datetime.datetime(2018, 5, 1, 0, 0), 'The Hyde', 1415, datetime.datetime(2019, 5, 1, 0, 0), 2952], [datetime.datetime(2018, 5, 1, 0, 0), 'The Landmark', 1406, datetime.datetime(2020, 11, 1, 0, 0), 2162], [datetime.datetime(2018, 5, 1, 0, 0), 'Chancery Court', 1610, None, -1], [datetime.datetime(2018, 5, 1, 0, 0), 'Peak Residence', 1558, datetime.datetime(2021, 4, 1, 0, 0), 2527], [datetime.datetime(2018, 4, 1, 0, 0), 'Avenue South Residence', 1138, datetime.datetime(2019, 9, 1, 0, 0), 2056], [datetime.datetime(2018, 5, 1, 0, 0), 'The Antares', 1109, datetime.datetime(2019, 9, 1, 0, 0), 1782], [datetime.datetime(2018, 5, 1, 0, 0), 'Cuscaden Reserve', 2377, datetime.datetime(2019, 9, 1, 0, 0), 3604], [datetime.datetime(2018, 4, 1, 0, 0), '15 Holland Hill', 1712, datetime.datetime(2020, 4, 1, 0, 0), 2995], [datetime.datetime(2018, 4, 1, 0, 0), 'Sky Everton', 1565, datetime.datetime(2019, 6, 1, 0, 0), 2598], [datetime.datetime(2018, 4, 1, 0, 0), 'Leedon Green', 1790, datetime.datetime(2020, 1, 1, 0, 0), 2672], [datetime.datetime(2018, 4, 1, 0, 0), 'Katong Omega Apartments', 1141, None, -1], [datetime.datetime(2018, 4, 1, 0, 0), 'Hyll on Holland', 1504, datetime.datetime(2020, 10, 1, 0, 0), 2446], [datetime.datetime(2018, 3, 1, 0, 0), 'Fairhaven', 1629, None, -1], [datetime.datetime(2018, 3, 1, 0, 0), 'The Avenir', 1806, datetime.datetime(2020, 1, 1, 0, 0), 3185], [datetime.datetime(2018, 3, 1, 0, 0), 'Jervois Treasures', 1683, datetime.datetime(2020, 2, 1, 0, 0), 2389], [datetime.datetime(2018, 3, 1, 0, 0), 'Katong Park Towers', 1280, None, -1], [datetime.datetime(2018, 3, 1, 0, 0), 'Urban Treasures', 1118, datetime.datetime(2019, 11, 1, 0, 0), 1958], [datetime.datetime(2018, 3, 1, 0, 0), 'The Atelier', 1626, datetime.datetime(2021, 3, 1, 0, 0), 2751], [datetime.datetime(2018, 3, 1, 0, 0), 'Hyll on Holland', 1703, datetime.datetime(2020, 10, 1, 0, 0), 2446], [datetime.datetime(2018, 3, 1, 0, 0), 'Piermont Grand', 583, datetime.datetime(2019, 7, 1, 0, 0), 1110], [datetime.datetime(2018, 2, 1, 0, 0), 'City Towers', 1847, None, -1], [datetime.datetime(2018, 2, 1, 0, 0), 'Whistler Grand', 800, datetime.datetime(2018, 11, 1, 0, 0), 1412], [datetime.datetime(2018, 2, 1, 0, 0), 'Haus on Handy', 1722, datetime.datetime(2019, 7, 1, 0, 0), 2813], [datetime.datetime(2018, 1, 1, 0, 0), 'The Wilshire', 1536, None, -1], [datetime.datetime(2018, 3, 1, 0, 0), 'Ampas Apartments', 1073, None, -1], [datetime.datetime(2018, 1, 1, 0, 0), 'View at Kismis', 855, datetime.datetime(2019, 7, 1, 0, 0), 1703], [datetime.datetime(2018, 2, 1, 0, 0), 'Riviera Point', 1461, None, -1], [datetime.datetime(2017, 12, 1, 0, 0), 'Fyve Derbyshire', 1390, datetime.datetime(2019, 1, 1, 0, 0), 2320], [datetime.datetime(2017, 12, 1, 0, 0), 'Apartment 8', 1061, None, -1], [datetime.datetime(2018, 3, 1, 0, 0), 'Rezi 24', 818, datetime.datetime(2019, 3, 1, 0, 0), 1579], [datetime.datetime(2017, 12, 1, 0, 0), 'Jervois Green', 1601, None, -1], [datetime.datetime(2017, 12, 1, 0, 0), 'Royalgreen', 1960, datetime.datetime(2019, 10, 1, 0, 0), 2730], [datetime.datetime(2017, 11, 1, 0, 0), 'The Hyde', 1527, datetime.datetime(2019, 5, 1, 0, 0), 2952], [datetime.datetime(2017, 11, 1, 0, 0), 'Lodge 77', 902, None, -1], [datetime.datetime(2017, 12, 1, 0, 0), 'Crystal Tower', 1840, None, -1], [datetime.datetime(2017, 11, 1, 0, 0), 'The Lilium', 1092, datetime.datetime(2019, 6, 1, 0, 0), 2032], [datetime.datetime(2017, 11, 1, 0, 0), 'Tai Wah Building', 2209, None, -1], [datetime.datetime(2017, 11, 1, 0, 0), '35 Gilstead', 1638, datetime.datetime(2019, 3, 1, 0, 0), 2517], [datetime.datetime(2017, 10, 1, 0, 0), 'Amber Park', 1515, datetime.datetime(2019, 5, 1, 0, 0), 2468], [datetime.datetime(2017, 10, 1, 0, 0), 'Dunearn Court', 1371, None, -1], [datetime.datetime(2017, 10, 1, 0, 0), 'Florence Regency', 842, None, -1], [datetime.datetime(2017, 10, 1, 0, 0), 'Changi Garden', 888, None, -1], [datetime.datetime(2017, 10, 1, 0, 0), 'Normanton Park', 969, datetime.datetime(2021, 1, 1, 0, 0), 1785], [datetime.datetime(2017, 9, 1, 0, 0), 'Nanak Mansions', 1429, None, -1], [datetime.datetime(2017, 9, 1, 0, 0), 'Jervois Gardens', 1511, None, -1], [datetime.datetime(2017, 9, 1, 0, 0), 'Nyon', 1222, datetime.datetime(2019, 3, 1, 0, 0), 2256], [datetime.datetime(2017, 9, 1, 0, 0), 'The Gazania', 1325, datetime.datetime(2019, 5, 1, 0, 0), 2042], [datetime.datetime(2017, 10, 1, 0, 0), 'Arena Residences', 781, datetime.datetime(2018, 11, 1, 0, 0), 1801], [datetime.datetime(2017, 8, 1, 0, 0), 'Sloane Residences', 1616, datetime.datetime(2019, 6, 1, 0, 0), 2890], [datetime.datetime(2018, 5, 1, 0, 0), "Villa D'Este", 1898, None, -1], [datetime.datetime(2018, 1, 1, 0, 0), 'Parc Clematis', 850, datetime.datetime(2019, 8, 1, 0, 0), 1623], [datetime.datetime(2017, 8, 1, 0, 0), 'Treasure at Tampines', 676, datetime.datetime(2019, 3, 1, 0, 0), 1356], [datetime.datetime(2017, 8, 1, 0, 0), 'Parkwood Residences', 424, datetime.datetime(2020, 11, 1, 0, 0), 1434], [datetime.datetime(2017, 8, 1, 0, 0), 'The Garden Residences', 965, datetime.datetime(2018, 6, 1, 0, 0), 1577], [datetime.datetime(2017, 7, 1, 0, 0), 'Affinity at Serangoon', 835, datetime.datetime(2018, 6, 1, 0, 0), 1494], [datetime.datetime(2017, 7, 1, 0, 0), 'The Albracca', 1409, None, -1], [datetime.datetime(2017, 7, 1, 0, 0), 'The Verandah Residences', 964, datetime.datetime(2018, 4, 1, 0, 0), 1794], [datetime.datetime(2017, 7, 1, 0, 0), 'Park Colonial', 1110, datetime.datetime(2018, 7, 1, 0, 0), 1760], [datetime.datetime(2017, 6, 1, 0, 0), 'Parc Esta', 909912000, datetime.datetime(2018, 11, 1, 0, 0), 1683], [datetime.datetime(2017, 5, 1, 0, 0), 'Stirling Residences', 1051, datetime.datetime(2018, 7, 1, 0, 0), 1828], [datetime.datetime(2017, 5, 1, 0, 0), 'The Tapestry', 565, datetime.datetime(2018, 3, 1, 0, 0), 1340], [datetime.datetime(2017, 2, 1, 0, 0), 'Twin Vew', 592, datetime.datetime(2018, 5, 1, 0, 0), 1401], [datetime.datetime(2017, 1, 1, 0, 0), 'Amber 45', 1063, datetime.datetime(2018, 5, 1, 0, 0), 2294], [datetime.datetime(2017, 1, 1, 0, 0), 'Bukit 828', 663, datetime.datetime(2018, 9, 1, 0, 0), 1601], [datetime.datetime(2016, 12, 1, 0, 0), 'Margaret Ville', 998, datetime.datetime(2018, 6, 1, 0, 0), 1836], [datetime.datetime(2016, 12, 1, 0, 0), 'Jui Residences', 2197, datetime.datetime(2018, 9, 1, 0, 0), 1758], [datetime.datetime(2016, 11, 1, 0, 0), '3 Cuscaden', 1826, datetime.datetime(2018, 11, 1, 0, 0), 3626], [datetime.datetime(2016, 11, 1, 0, 0), '120 Grange', 1841, datetime.datetime(2018, 6, 1, 0, 0), 3078], [datetime.datetime(2016, 11, 1, 0, 0), '8 Hullet', 2073, datetime.datetime(2018, 3, 1, 0, 0), 3445], [datetime.datetime(2016, 10, 1, 0, 0), 'The Tre Ver', 797, datetime.datetime(2018, 8, 1, 0, 0), 1598], [datetime.datetime(2016, 9, 1, 0, 0), 'Parc Botannia', 517, datetime.datetime(2017, 11, 1, 0, 0), 1303], [datetime.datetime(2016, 9, 1, 0, 0), 'Rivercove Residences', 355, datetime.datetime(2018, 4, 1, 0, 0), 980], [datetime.datetime(2016, 9, 1, 0, 0), 'Rivercove Residences', 355, datetime.datetime(2018, 4, 1, 0, 0), 980], [datetime.datetime(2016, 8, 1, 0, 0), 'HARBOUR VIEW GARDENS', 1082, datetime.datetime(2018, 4, 1, 0, 0), 1748], [datetime.datetime(2016, 7, 1, 0, 0), 'Martin Modern', 1239, datetime.datetime(2017, 7, 1, 0, 0), 2498], [datetime.datetime(2016, 4, 1, 0, 0), 'Kandis Residence', 481, datetime.datetime(2017, 9, 1, 0, 0), 1265], [datetime.datetime(2016, 5, 1, 0, 0), 'JadeScape', 747, datetime.datetime(2018, 9, 1, 0, 0), 1704], [datetime.datetime(2016, 2, 1, 0, 0), 'Grandeur Park Residences', 761, datetime.datetime(2017, 3, 1, 0, 0), 1403], [datetime.datetime(2016, 2, 1, 0, 0), 'Hundred Palms Residences', 331, datetime.datetime(2017, 7, 1, 0, 0), 849], [datetime.datetime(2016, 1, 1, 0, 0), 'Seaside Residences', 858, datetime.datetime(2017, 4, 1, 0, 0), 1764], [datetime.datetime(2015, 12, 1, 0, 0), 'The Clement Canopy', 615, datetime.datetime(2017, 2, 1, 0, 0), 1375], [datetime.datetime(2015, 11, 1, 0, 0), 'Artra', 851, datetime.datetime(2017, 4, 1, 0, 0), 1774], [datetime.datetime(2015, 11, 1, 0, 0), 'Forest Woods', 710, datetime.datetime(2017, 1, 1, 0, 0), 1457], [datetime.datetime(2015, 8, 1, 0, 0), 'Parc Riviera', 551, datetime.datetime(2017, 1, 1, 0, 0), 1228], [datetime.datetime(2015, 8, 1, 0, 0), 'Inz Residence', 295, datetime.datetime(2017, 3, 1, 0, 0), 796], [datetime.datetime(2015, 6, 1, 0, 0), 'Queens Peak', 871, datetime.datetime(2017, 1, 1, 0, 0), 1687], [datetime.datetime(2015, 6, 1, 0, 0), 'Gem Residences', 755, datetime.datetime(2017, 2, 1, 0, 0), 1534], [datetime.datetime(2015, 5, 1, 0, 0), 'The Alps Residences', 483, datetime.datetime(2017, 1, 1, 0, 0), 1083], [datetime.datetime(2015, 3, 1, 0, 0), 'Sturdee Residences', 787, datetime.datetime(2017, 2, 1, 0, 0), 1534], [datetime.datetime(2015, 3, 1, 0, 0), 'Lake Grande', 630, datetime.datetime(2017, 1, 1, 0, 0), 1317], [datetime.datetime(2015, 2, 1, 0, 0), 'Northwave', 278, datetime.datetime(2017, 1, 1, 0, 0), 795], [datetime.datetime(2015, 2, 1, 0, 0), 'TREASURE CREST', 280, datetime.datetime(2017, 1, 1, 0, 0), 851], [datetime.datetime(2014, 11, 1, 0, 0), 'STARS OF KOVAN', 849, datetime.datetime(2017, 1, 1, 0, 0), 1456], [datetime.datetime(2014, 10, 1, 0, 0), 'THOMSON IMPRESSIONS', 731, datetime.datetime(2017, 1, 1, 0, 0), 1438], [datetime.datetime(2014, 10, 1, 0, 0), 'THE VISIONAIRE', 353, datetime.datetime(2017, 1, 1, 0, 0), 820], [datetime.datetime(2014, 8, 1, 0, 0), 'High Park Residences', 448, datetime.datetime(2017, 2, 1, 0, 0), 996], [datetime.datetime(2014, 8, 1, 0, 0), 'High Park Residences', 438, datetime.datetime(2017, 2, 1, 0, 0), 996], [datetime.datetime(2014, 7, 1, 0, 0), 'Parc Life', 320, datetime.datetime(2017, 1, 1, 0, 0), 807], [datetime.datetime(2014, 5, 1, 0, 0), 'Signature at Yishun', 334, datetime.datetime(2017, 1, 1, 0, 0), 779], [datetime.datetime(2014, 5, 1, 0, 0), 'The Criterion', 330, datetime.datetime(2017, 1, 1, 0, 0), 780], [datetime.datetime(2014, 4, 1, 0, 0), 'Sims Urban Oasis', 688, datetime.datetime(2017, 1, 1, 0, 0), 1417], [datetime.datetime(2014, 4, 1, 0, 0), 'Principal Garden', 821, datetime.datetime(2017, 1, 1, 0, 0), 1703], [datetime.datetime(2014, 3, 1, 0, 0), 'Symphony Suites', 450, datetime.datetime(2017, 1, 1, 0, 0), 1059], [datetime.datetime(2014, 3, 1, 0, 0), 'Sol Acres', 339, datetime.datetime(2017, 1, 1, 0, 0), 796], [datetime.datetime(2014, 3, 1, 0, 0), 'Sol Acres', 375, datetime.datetime(2017, 1, 1, 0, 0), 796], [datetime.datetime(2014, 2, 1, 0, 0), 'The Vales', 367, datetime.datetime(2017, 1, 1, 0, 0), 834], [datetime.datetime(2014, 1, 1, 0, 0), 'The Brownstone', 350, datetime.datetime(2017, 2, 1, 0, 0), 826], [datetime.datetime(2014, 1, 1, 0, 0), 'TRE Residences', 776, datetime.datetime(2017, 3, 1, 0, 0), 1428], [datetime.datetime(2014, 1, 1, 0, 0), 'Botanique At Bartley', 648, datetime.datetime(2017, 2, 1, 0, 0), 1304], [datetime.datetime(2014, 1, 1, 0, 0), 'Westwood Residences', 382, datetime.datetime(2017, 1, 1, 0, 0), 796], [datetime.datetime(2013, 12, 1, 0, 0), 'Kingsford Waterbay', 522, datetime.datetime(2017, 1, 1, 0, 0), 1272], [datetime.datetime(2013, 12, 1, 0, 0), 'Kingsford Waterbay', 522, datetime.datetime(2017, 1, 1, 0, 0), 1272], [datetime.datetime(2013, 9, 1, 0, 0), 'Sophia Hills', 1156, datetime.datetime(2017, 1, 1, 0, 0), 2015], [datetime.datetime(2013, 8, 1, 0, 0), 'Lake Life', 418, datetime.datetime(2017, 2, 1, 0, 0), 806], [datetime.datetime(2013, 8, 1, 0, 0), 'The Terrace', 355, datetime.datetime(2017, 1, 1, 0, 0), 784], [datetime.datetime(2013, 8, 1, 0, 0), 'The Amore', 356, datetime.datetime(2017, 2, 1, 0, 0), 769], [datetime.datetime(2013, 7, 1, 0, 0), 'The Santorini', 562, datetime.datetime(2017, 1, 1, 0, 0), 1030], [datetime.datetime(2013, 6, 1, 0, 0), 'Victoria Park Villas', 908, datetime.datetime(2017, 3, 1, 0, 0), 1778], [datetime.datetime(2013, 6, 1, 0, 0), 'Waterfront @ Faber', 687, None, -1], [datetime.datetime(2013, 6, 1, 0, 0), 'Rivertrees Residences', 533, datetime.datetime(2017, 2, 1, 0, 0), 1062], [datetime.datetime(2013, 5, 1, 0, 0), 'Bellewaters', 331, datetime.datetime(2017, 4, 1, 0, 0), 812], [datetime.datetime(2013, 5, 1, 0, 0), 'Bellewoods', 341, datetime.datetime(2017, 1, 1, 0, 0), 742], [datetime.datetime(2013, 4, 1, 0, 0), 'Highline Residences', 1163, datetime.datetime(2017, 1, 1, 0, 0), 1885], [datetime.datetime(2013, 4, 1, 0, 0), 'Riverbank @ Fernvale', 489, datetime.datetime(2017, 1, 1, 0, 0), 1006], [datetime.datetime(2013, 2, 1, 0, 0), 'Commonwealth Towers', 883, datetime.datetime(2017, 1, 1, 0, 0), 1696], [datetime.datetime(2013, 1, 1, 0, 0), 'Lakeville', 651, datetime.datetime(2017, 1, 1, 0, 0), 1189], [datetime.datetime(2013, 1, 1, 0, 0), 'The Panorama', 790, datetime.datetime(2017, 1, 1, 0, 0), 1311]]

for i in all:
    if (i[4] == -1) or (i[3] is None):
        print(i)
        all.remove(i)

for i in all:
    if (i[4] == -1) or (i[3] is None):
        print(i)
        all.remove(i)
        
for i in all:
    if (i[2] >= 5000):
        print(i)
        all.remove(i)

all_df = pd.DataFrame(all)
all_df.columns = ["land_txn_date", "proj_name", "land_txn_price", "openning_date", "openning_price"]
all_df.to_csv("land_transaction_data.csv")

[datetime.datetime(2019, 12, 1, 0, 0), 'Casa Sophia', 1200, None, -1]
[datetime.datetime(2018, 6, 1, 0, 0), 'Teck Guan Ville', 1300, None, -1]
[datetime.datetime(2018, 5, 1, 0, 0), 'Chancery Court', 1610, None, -1]
[datetime.datetime(2018, 4, 1, 0, 0), 'Katong Omega Apartments', 1141, None, -1]
[datetime.datetime(2018, 3, 1, 0, 0), 'Fairhaven', 1629, None, -1]
[datetime.datetime(2018, 3, 1, 0, 0), 'Katong Park Towers', 1280, None, -1]
[datetime.datetime(2018, 2, 1, 0, 0), 'City Towers', 1847, None, -1]
[datetime.datetime(2018, 1, 1, 0, 0), 'The Wilshire', 1536, None, -1]
[datetime.datetime(2018, 2, 1, 0, 0), 'Riviera Point', 1461, None, -1]
[datetime.datetime(2017, 12, 1, 0, 0), 'Apartment 8', 1061, None, -1]
[datetime.datetime(2017, 12, 1, 0, 0), 'Jervois Green', 1601, None, -1]
[datetime.datetime(2017, 11, 1, 0, 0), 'Lodge 77', 902, None, -1]
[datetime.datetime(2017, 11, 1, 0, 0), 'Tai Wah Building', 2209, None, -1]
[datetime.datetime(2017, 10, 1, 0, 0), 'Dunearn Court', 1371, None, 

# 5. Collate final newly launched projects dataset (need to predict) and save as csv file

In [3]:
def get_new_proj_data():
    all_projs = retrieve_from_html()
    new_projs = all_projs[:16]
    
    for i in new_projs:
        if i[2] == 0:
            print(i)
            new_projs.remove(i)
    
    df_new_projs_data = pd.DataFrame(new_projs)
    df_new_projs_data.columns = ["land_txn_date", "proj_name", "land_txn_price"]

    print(df_new_projs_data)
    df_new_projs_data.to_csv("new_proj_land_txn_data.csv", index=False)
    
get_new_proj_data()

[datetime.datetime(2021, 6, 1, 0, 0), 'Ji Liang Gardens', 0]
[datetime.datetime(2021, 2, 1, 0, 0), '10A/B &amp; 11 Institution Hills', 0]
   land_txn_date                  proj_name  land_txn_price
0     2022-01-01            Gloria Mansions            1098
1     2021-12-01          Former High Point            2626
2     2021-12-01            Former La Ville            1540
3     2021-10-01  Watten Estate Condominium            1723
4     2021-10-01         Slim Barracks Rise            1210
5     2021-10-01         Slim Barracks Rise            1246
6     2021-09-01                 Flynn Park            1318
7     2021-08-01         Tampines Street 62             659
8     2021-06-01         Tengah Garden Walk             603
9     2021-06-01        Ang Mo Kio Avenue 1            1118
10    2021-05-01  Northumberland Road Condo            1129
11    2021-07-01         6A/B/C Robin Drive            1454
12    2021-03-01   2, 4, 6 Mount Emily Road            1115
13    2021-03-01      